# Practical Aspects of Scientific Computing

> Your time is more valuable than computer time -- Everyone, ever

### Topics

We discuss several aspects of computing that are things you do but don't often think about.  They can potentially save a large amount of time for you (not your computer).  In particular, we are going to look at

* **Debugging** -- finding and fixing problems in your code
* **Versioning** -- maintaining a consistent set of code with history
* **Style** -- how you write your code influences its legibility
* **Structure** -- the structure of your code
* **Tools** -- choosing the right set of tools for the job

# Debugging

One of the most common things you do when creating a software package or solving a science problem is to debug your code.  I'm taking 'debug' in the sense of removing problems from your code.  So the natural place to begin is: how do you find bugs?

### Finding bugs / Tests

While the most natural way to find problems is to allow them to surface, this is not always the best practice.  Some issues are more subtle than you'd expect / appreciate at first glance.  That is why some people advocate for *test-driven* development. In this paradigm, you must write a battery of tests that cover the entire functionality of the code base.  

In [ ]:


Tests
Simplest case
Bisect print
ipdb, %debug, Tracer()
break points, old school